In [1]:
%pip install tqdm python-dotenv PyMuPDF

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\oheit\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import logging
import time
from tqdm import tqdm
from typing import Optional
from dotenv import load_dotenv
import json

In [3]:
from os import path, getenv
from sys import path as sys_path
parent_dir = path.abspath(path.join('..'))
if parent_dir not in sys_path:
    sys_path.append(parent_dir)
    print(f"Added {parent_dir.split("\\")[-1]} to sys.path")
from client.openUiClient import OpenUiClient

Added ai to sys.path


In [4]:
client = OpenUiClient()

In [5]:
models = [
    # 'deepseek-r1:14b', 
          'llama3:8b', 
        #   'qwen:1.8b', 
        #   'phi:latest',
          'llama3.1:8b', 
        #   'stable-code:latest', 
          #'gemma3:12b', 
          # 'phi4-reasoning:latest',
          #'llama3.2-vision:11b'
          ]

In [6]:
def read_json(file_path):
        """
        Read a JSON file and return its content.
        
        :param file_path: The path of the JSON file to read.
        :return: The content of the JSON file as a dictionary.
        """
        try:
            file_path
            with open(file_path, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            logging.error(f"File not found: {file_path}")
            return None
        except json.JSONDecodeError as e:
            logging.error(f"Error decoding JSON from {file_path}: {e}")
            return None

In [7]:
def test_chatWithModel(model: Optional[str] = 'llama3.1:8b'):
    filePath = path.abspath(path.join('..', '..', 'evaluate_set.json'))
    json_data = read_json(filePath)
    responses = []
    
    knowledge_file_1 = path.abspath(path.join('..','knowledge', 'the_corrective_commit_probability_code_quality_metric.pdf'))
    knowledge_file_2 = path.abspath(path.join('..','knowledge', 'what_makes_a_good_commit_message.pdf'))

    for index, item in json_data.items():
        repos = item
        for repo in tqdm(repos, desc='Processing repos', leave=False):
            main_content = []
            commits = repo.get('commits', [])

            for commit in tqdm(commits, desc='Processing commits', leave=False):
                commit_message = commit.get('message', '')
                files = commit.get('files_changed', [])
                sha = commit.get('sha', '')
                link = commit.get('link', '')
                main_content.append((commit_message, files, sha, link))
            
            for content in main_content:
                chat_response = client.evaluateCommitQualityChatWithModel(knowledge_files=[knowledge_file_1, knowledge_file_2], commit_msg=content[0], commit_files_changed=content[1], model=model)
                if chat_response:
                    print("Status code:", chat_response.status_code)
                    try:
                        chat_response_message = chat_response.json()['choices'][0]['message']['content']
                        print("Chat response:\n", chat_response_message)
                        responses.append({
                            'model': model,
                            'sha': content[2],
                            'link': content[3],
                            'response': chat_response_message,
                        })
                    except (KeyError, IndexError) as e:
                        print(f"Error processing the chat response. Details: {e}")
                else:
                    print("Unable to perform the chat.")
                time.sleep(0.5)
    return responses

In [8]:
import json

def save_output(output: list, file_path: str):
    """
    Save the structured output of model(s) to a JSON file.

    The output will be structured as a dictionary with the model name as the key,
    and a list of objects each containing the SHA, link, and parsed JSON response.

    Args:
        output (list): List of dictionaries with keys: 'model', 'sha', 'link', and 'response' (stringified JSON).
        file_path (str): Path to the output file.
    """
    output_dict = {}

    for response in output:
        model = response['model']
        sha = response['sha']
        link = response['link']
        raw_json = response['response']

        try:
            parsed_response = json.loads(raw_json)
        except json.JSONDecodeError as e:
            print(f"Invalid JSON in response for commit {sha}. Error: {e}")
            parsed_response = {"parse_error": str(e), "raw": raw_json}

        # Organize results by model
        if model not in output_dict:
            output_dict[model] = []

        output_dict[model].append({
            'sha': sha,
            'link': link,
            'response': parsed_response
        })

    # Write to file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(output_dict, file, indent=4)
    
    print(f"Output saved to {file_path}")


In [9]:
print(client.getAvailableApiModels())

['deepseek-r1:14b', 'llama3:8b', 'llama3.1:8b', 'llama3.2-vision:11b', 'medllama2:7b', 'phi:latest', 'qwen:0.5b', 'qwen:1.8b', 'qwen:4b', 'stable-code:3b', 'stable-code:latest', 'gemma3:12b', 'gemma3:1b', 'gemma3:4b', 'phi4:14b', 'qwen2.5:14b', 'phi4-reasoning:latest']


In [ ]:
output_stable_code = test_chatWithModel()
save_output(output_stable_code, path.abspath(path.join('..','..','evaluation_for_llama3_model.json')))

Processing repos:   0%|          | 0/4 [00:00<?, ?it/s]

Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": false,
  "why_present": true,
  "files_changed": 17
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": false,
  "why_present": true,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 4
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 2
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": true,
  "files_changed": 2
}
Status code: 200
Chat response:
 {
  "what_pre

Processing repos:  25%|██▌       | 1/4 [03:24<10:14, 204.88s/it]

Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 6
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 3
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 2
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 27
}
Status code: 200
Chat response:
 {
  "what_present": false,
  "why_present": true,
  "files_changed": 12
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 27
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 3
}
Status code: 200
Chat response:
 {
  "what_

Processing repos:  50%|█████     | 2/4 [06:48<06:48, 204.38s/it]

Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": true,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": false,
  "why_present": true,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": false,
  "why_present": false,
  "files_changed": 4
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 4
}
Status code: 200
Chat response:
 {
  "what_pre

Processing repos:  75%|███████▌  | 3/4 [10:13<03:24, 204.52s/it]

Status code: 200
Chat response:
 {
  "what_present": false,
  "why_present": false,
  "files_changed": 4
}
Status code: 200
Chat response:
 {
  "what_present": false,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 2
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 5
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_present": true,
  "why_present": false,
  "files_changed": 1
}
Status code: 200
Chat response:
 {
  "what_p

Output saved to c:\Users\oheit\Desktop\UNICAMP\IC code samples\projs\finer\evaluation_for_llama3_model.json


: 